In [1]:
import plotly
import plotly.io as pio
import plotly.graph_objects as go
import chart_studio.plotly as py
pio.renderers.default = "vscode"
import pandas as pd
import numpy as np
# print out all numpy array
np.set_printoptions(threshold=np.inf)

from scipy.interpolate import griddata

from itertools import product

from sklearn import datasets
from sklearn.gaussian_process import GaussianProcessRegressor

In [2]:
# load the dataset
data = pd.read_excel(
    "240905 Simulated Automated Screening Yields for DOE Practice.xlsx"
)
data

,Phase,Temperature (°C),Rotation (RPM),Current (mA),Sim. Yields
0,1,20.000000,150.000000,20.000000,80
1,2,60.000000,150.000000,20.000000,90
2,3,20.000000,600.000000,20.000000,83
3,4,60.000000,600.000000,20.000000,93
4,5,20.000000,150.000000,100.000000,20
5,6,60.000000,150.000000,100.000000,30
6,7,20.000000,600.000000,100.000000,23
7,8,60.000000,600.000000,100.000000,33
8,9,40.000000,375.000000,60.000000,56
9,10,6.364143,375.000000,60.000000,41


In [3]:
header = data.columns
data = data.to_numpy()
data = data[:, 1:]

In [4]:
# find the max value of the 'Sim. Yields' column
max_yield = np.argmax(data[:, 3])

In [5]:
# Original data plot
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=data[:, 0],
            y=data[:, 1],
            z=data[:, 3],
            mode="markers",
            marker=dict(size=6, color="blue", opacity=0.8, symbol="circle"),
            name="Data Points",
        )
    ]
)

# Highlight the highest yield with red
fig.add_trace(
    go.Scatter3d(
        x=[data[max_yield, 0]],
        y=[data[max_yield, 1]],
        z=[data[max_yield, 3]],
        mode="markers",
        marker=dict(size=10, color="red", symbol="diamond", opacity=1.0),
        name="Highest Yield",
    )
)

# Improve layout
fig.update_layout(
    title="3D Scatter Plot of Yield vs Temperature and Rotation",
    scene=dict(
        xaxis=dict(title="Temperature (°C)", showbackground=True, backgroundcolor="lightblue"),
        yaxis=dict(title="Rotation (RPM)", showbackground=True, backgroundcolor="lightyellow"),
        zaxis=dict(title="Yield", showbackground=True, backgroundcolor="lightgreen"),
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    legend=dict(x=0.01, y=0.99, bgcolor="rgba(255, 255, 255, 0.5)", bordercolor="black", borderwidth=1),
    template="plotly_white",  # or use "plotly_white" or any other built-in template
)

# Save to HTML file
pio.write_html(fig, file='2024-09-11-original_data.html', auto_open=True)

fig.show()

In [6]:
def bounds(data) -> list:
    mins = np.min(data, axis=0)
    maxs = np.max(data, axis=0)
    return [[mins_, maxs_] for mins_, maxs_ in zip(mins, maxs)]

# create the line space
def fullfact(bound_array, num_levels: int) -> np.ndarray:
    return np.array(list(product(*[np.linspace(min_, max_, num_levels) for min_, max_ in bound_array])))

In [7]:
data[:, 0:3]

array([[ 20.        , 150.        ,  20.        ],
       [ 60.        , 150.        ,  20.        ],
       [ 20.        , 600.        ,  20.        ],
       [ 60.        , 600.        ,  20.        ],
       [ 20.        , 150.        , 100.        ],
       [ 60.        , 150.        , 100.        ],
       [ 20.        , 600.        , 100.        ],
       [ 60.        , 600.        , 100.        ],
       [ 40.        , 375.        ,  60.        ],
       [  6.36414339, 375.        ,  60.        ],
       [ 40.        , 753.40338686,  60.        ],
       [ 40.        ,  -3.40338686,  60.        ],
       [ 40.        , 375.        , 127.27171322],
       [ 73.63585661, 375.        ,  60.        ]])

In [8]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# Define a kernel with different parameters
kernel = C(1.0, (1e-3, 1e4)) * RBF(10, (1e-2, 1e2))

# Create the Gaussian Process Regressor with a regularization term
regressor = GaussianProcessRegressor(kernel=kernel, alpha=1e-2, random_state=42)

# Fit the model
regressor.fit(data[:, 0:3], data[:, 3])

# Print the kernel and score
print(f"Kernel: {regressor.kernel_}")
print(f"Log Marginal Likelihood: {regressor.log_marginal_likelihood_value_}")

Kernel: 47.2**2 * RBF(length_scale=100)
Log Marginal Likelihood: -64.84048299456826


c:\Users\R9000P\EChem_Auto_Potential_Optimization\venv\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.



In [9]:
# predict the yield
line_space = fullfact(bounds(data[:, 0:3]), 20)
line_space

array([[  6.36414339,  -3.40338686,  20.        ],
       [  6.36414339,  -3.40338686,  25.64587964],
       [  6.36414339,  -3.40338686,  31.29175929],
       [  6.36414339,  -3.40338686,  36.93763893],
       [  6.36414339,  -3.40338686,  42.58351857],
       [  6.36414339,  -3.40338686,  48.22939822],
       [  6.36414339,  -3.40338686,  53.87527786],
       [  6.36414339,  -3.40338686,  59.5211575 ],
       [  6.36414339,  -3.40338686,  65.16703715],
       [  6.36414339,  -3.40338686,  70.81291679],
       [  6.36414339,  -3.40338686,  76.45879643],
       [  6.36414339,  -3.40338686,  82.10467607],
       [  6.36414339,  -3.40338686,  87.75055572],
       [  6.36414339,  -3.40338686,  93.39643536],
       [  6.36414339,  -3.40338686,  99.042315  ],
       [  6.36414339,  -3.40338686, 104.68819465],
       [  6.36414339,  -3.40338686, 110.33407429],
       [  6.36414339,  -3.40338686, 115.97995393],
       [  6.36414339,  -3.40338686, 121.62583358],
       [  6.36414339,  -3.40338

In [10]:
predicted_yield, std = regressor.predict(line_space, return_std=True)

In [11]:
predicted_yield

array([ 5.01306577e+01,  4.99125826e+01,  4.95138582e+01,  4.89371688e+01,
        4.81870472e+01,  4.72698166e+01,  4.61935029e+01,  4.49677174e+01,
        4.36035148e+01,  4.21132273e+01,  4.05102786e+01,  3.88089838e+01,
        3.70243357e+01,  3.51717859e+01,  3.32670218e+01,  3.13257462e+01,
        2.93634632e+01,  2.73952737e+01,  2.54356854e+01,  2.34984400e+01,
        6.09820351e+01,  6.00917940e+01,  5.89637019e+01,  5.76064638e+01,
        5.60312901e+01,  5.42517437e+01,  5.22835457e+01,  5.01443453e+01,
        4.78534559e+01,  4.54315655e+01,  4.29004263e+01,  4.02825292e+01,
        3.76007718e+01,  3.48781258e+01,  3.21373096e+01,  2.94004747e+01,
        2.66889102e+01,  2.40227717e+01,  2.14208388e+01,  1.89003056e+01,
        6.96145694e+01,  6.79093845e+01,  6.59123423e+01,  6.36393513e+01,
        6.11094690e+01,  5.83446371e+01,  5.53693652e+01,  5.22103699e+01,
        4.88961753e+01,  4.54566850e+01,  4.19227337e+01,  3.83256267e+01,
        3.46966800e+01,  

In [12]:
line_space[:, 0]

array([ 6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36414339,
        6.36414339,  6.36414339,  6.36414339,  6.36414339,  6.36

In [13]:
import plotly.graph_objs as go
import plotly.io as pio

# Assuming `predicted_yield` is obtained from the previous step and is of the same length as `line_space`

# Original data plot
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=data[:, 0],
            y=data[:, 1],
            z=data[:, 3],
            mode="markers",
            marker=dict(size=6, color="blue", opacity=0.8, symbol="circle"),
            name="Original Data Points",
        )
    ]
)

# Highlight the highest yield with red
max_yield = np.argmax(data[:, 3])
fig.add_trace(
    go.Scatter3d(
        x=[data[max_yield, 0]],
        y=[data[max_yield, 1]],
        z=[data[max_yield, 3]],
        mode="markers",
        marker=dict(size=10, color="red", symbol="diamond", opacity=1.0),
        name="Highest Yield",
    )
)

# Predicted data plot
fig.add_trace(
    go.Scatter3d(
        x=line_space[:, 0],
        y=line_space[:, 1],
        z=predicted_yield.flatten(),
        mode="markers",
        marker=dict(size=4, color="green", opacity=0.5, symbol="circle"),
        name="Predicted Data Points",
    )
)

# Improve layout
fig.update_layout(
    title="Yield vs Temperature and Rotation after Gaussian Regression",
    scene=dict(
        xaxis=dict(title="Temperature (°C)", showbackground=True, backgroundcolor="lightblue"),
        yaxis=dict(title="Rotation (RPM)", showbackground=True, backgroundcolor="lightyellow"),
        zaxis=dict(title="Yield", showbackground=True, backgroundcolor="lightgreen"),
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    legend=dict(x=0.01, y=0.99, bgcolor="rgba(255, 255, 255, 0.5)", bordercolor="black", borderwidth=1),
    template="plotly_white",  # or use "plotly_white" or any other built-in template
)

# Save to HTML file
pio.write_html(fig, file='2024-09-11-predicted_vs_original_data.html', auto_open=True)

# Show the plot
fig.show()

In [14]:
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio
from collections import defaultdict

# Assuming line_space and predicted_yield are as previously defined

# Create a dictionary to store the highest yield for each (x, y) pair
filtered_points = defaultdict(lambda: float('-inf'))

# Iterate over the points to keep the highest yield for each (x, y) pair
for i in range(len(line_space)):
    x, y, z = line_space[i, 0], line_space[i, 1], predicted_yield[i]
    if z > filtered_points[(x, y)]:
        filtered_points[(x, y)] = z

# Extract the filtered data
filtered_x = []
filtered_y = []
filtered_z = []

for (x, y), z in filtered_points.items():
    filtered_x.append(x)
    filtered_y.append(y)
    filtered_z.append(z)

# Original data plot
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=data[:, 0],
            y=data[:, 1],
            z=data[:, 3],
            mode="markers",
            marker=dict(size=6, color="blue", opacity=0.8, symbol="circle"),
            name="Original Data Points",
        )
    ]
)

# Highlight the highest yield with red
max_yield = np.argmax(data[:, 3])
fig.add_trace(
    go.Scatter3d(
        x=[data[max_yield, 0]],
        y=[data[max_yield, 1]],
        z=[data[max_yield, 3]],
        mode="markers",
        marker=dict(size=10, color="red", symbol="diamond", opacity=1.0),
        name="Highest Yield",
    )
)

# Filtered predicted data plot
fig.add_trace(
    go.Scatter3d(
        x=filtered_x,
        y=filtered_y,
        z=filtered_z,
        mode="markers",
        marker=dict(size=4, color="green", opacity=0.5, symbol="circle"),
        name="Filtered Predicted Data Points",
    )
)

# Improve layout
fig.update_layout(
    title="highest yield for each (x, y) pair",
    scene=dict(
        xaxis=dict(title="Temperature (°C)", showbackground=True, backgroundcolor="lightblue"),
        yaxis=dict(title="Rotation (RPM)", showbackground=True, backgroundcolor="lightyellow"),
        zaxis=dict(title="Yield", showbackground=True, backgroundcolor="lightgreen"),
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    legend=dict(x=0.01, y=0.99, bgcolor="rgba(255, 255, 255, 0.5)", bordercolor="black", borderwidth=1),
    template="plotly_white",
)

# Save to HTML file
pio.write_html(fig, file='2024-09-11-filtered_predicted_vs_original_data.html', auto_open=True)

# Show the plot
fig.show()

In [15]:
# Create grid data for surface plot
grid_x, grid_y = np.meshgrid(
    np.linspace(min(filtered_x), max(filtered_x), 100),
    np.linspace(min(filtered_y), max(filtered_y), 100),
)

# Interpolate the z values onto the grid
grid_z = griddata(
    (filtered_x, filtered_y), filtered_z, (grid_x, grid_y), method="cubic"
)

# Create the surface plot
surface_plot = go.Figure(
    data=[
        go.Surface(
            x=grid_x,
            y=grid_y,
            z=grid_z,
            showscale=False
        ),
        go.Scatter3d(
            x=[data[max_yield, 0]],
            y=[data[max_yield, 1]],
            z=[data[max_yield, 3]],
            mode="markers",
            marker=dict(size=10, color="red", symbol="diamond", opacity=1.0),
            name="Highest Yield",
        ),
        go.Scatter3d(
            x=data[:, 0],
            y=data[:, 1],
            z=data[:, 3],
            mode="markers",
            marker=dict(size=6, color="blue", opacity=0.8, symbol="circle"),
            name="Original Data Points",
        )
    ]
)

# Improve layout
surface_plot.update_layout(
    title="Yield Surface Plot with Original Data Points",
    scene=dict(
        xaxis=dict(
            title="Temperature (°C)", showbackground=True, backgroundcolor="lightblue"
        ),
        yaxis=dict(
            title="Rotation (RPM)", showbackground=True, backgroundcolor="lightyellow"
        ),
        zaxis=dict(title="Yield", showbackground=True, backgroundcolor="lightgreen"),
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    template="plotly_white",
    legend=dict(
        x=0.02,  # Position the legend away from the color bar
        y=0.98,
        bgcolor="rgba(255, 255, 255, 0.5)",
        bordercolor="black",
        borderwidth=1
    )
)

# Save to HTML file
pio.write_html(
    surface_plot,
    file="2024-09-11-yield_surface_with_original_data.html",
    auto_open=True,
)

# Show the plot
surface_plot.show()